<a href="https://colab.research.google.com/github/ewertonhseixas/DnD-Master-Assistent-AI/blob/main/alura%2Bgoogle_gemini_projeto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
import textwrap # Para formatar melhor a saída de texto
from IPython.display import HTML, display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [35]:
##########################################
# --- Agente 1: Mestre Narrativo --- #
##########################################
def agente_narrador(tema, n_jogadores, dificuldade):

    narrador = Agent(
        name="agente_narrador",
        model="gemini-2.0-flash-thinking-exp",
        instruction="""
        Você é o Mestre Narrativo focado em Dungeons and Dragons 5E, é o coração criativo da one-shot, responsável
        por tecer a trama da aventura a partir das informações iniciais fornecidas
        (número de jogadores, tema, nível de dificuldade). Você conceberá
        uma história épica e com leve tom de humor quando necessário, definirá os locais a serem explorados, criará variados NPCs com
        suas motivações e diálogos (NÃO USAR O NOME ELARA), e estruturará a progressão da aventura
        em até 4 encontros distintos, culminando em um final que deixe portas
        abertas para o futuro.
        Suas responsabilidades são:
        Receber as informações iniciais: número de jogadores,
        tema central da aventura e nível de dificuldade desejado.
        Criar a história da one-shot, incluindo:
        Uma introdução envolvente para situar os jogadores.
        Até 4 encontros distintos, que podem ser combates, desafios de
        perícia, interações sociais ou enigmas.
        Descrição detalhadas dos locais importantes, NPCs chave (com suas personalidades
        e motivações), e objetos relevantes para a trama.
        Um final aberto que deixe espaço para possíveis continuações.
        Definir os objetivos principais dos jogadores dentro da aventura.
        """,
        description="Agente narrativo que cria a história e seus desafios",
        tools=[google_search]
    )

    entrada_do_agente_narrador = f"Tema: {tema}\nNúmero de Jogadores: {n_jogadores}\nNível de Dificuldade: {dificuldade}"

    historia = call_agent(narrador, entrada_do_agente_narrador)
    return historia

In [37]:
################################################
# --- Agente 2: Mestre de Encontros (Balanceamento) --- #
################################################
def agente_balanceador(n_jogadores, dificuldade, historia_inicial):
    balanceador = Agent(
        name="agente_balanceador",
        model="gemini-2.0-flash-thinking-exp",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é o Mestre de Encontros atuará como o balanceador de desafios da
        aventura, pegando a visão narrativa traduzindo-a em encontros práticos
        e adequados ao nível do grupo de jogadores. Para combates, selecionará e
        ajustará criaturas do bestiário, além de calcular o nível de ameaça do encontro.
        Para outros desafios, definirá as perícias necessárias, as CDs e as possíveis consequências,
        garantindo que cada obstáculo seja desafiador, mas não intransponível.
        Responsabilidades:
        Receber a história inicial e a informação sobre o número de jogadores
        e o Nível de Dificuldade.
        Analisar cada encontro proposto.
        Para encontros de combate:
        Selecionar criaturas apropriadas do bestiário de D&D 5e.
        Ajustar o número de criaturas para garantir um desafio adequado ao grupo (considerando o nível de dificuldade desejado).
        Calcular o Nível de Encontro (Encounter Level - EL) ou o Valor de Pontos
        de Experiência (XP Thresholds) para cada combate, garantindo que esteja
        dentro da faixa desejada de dificuldade (Fácil, Médio, Difícil, Mortal).
        Descrever o ambiente do combate, incluindo quaisquer elementos interativos ou perigos.
        Descrever armadilhas e obstáculos do terreno.
        Para outros tipos de encontros (enigmas, desafios de perícia, interações sociais):
        Definir os testes de perícia necessários e suas respectivas CD (Classe de Dificuldade).
        Sugerir possíveis consequências para sucesso e falha.
        No caso de enigmas, fornecer a descrição do enigma e as possíveis soluções.
        """,
        description="Agente que balancea todos os desafios da aventura para a quantidade de jogadores e Nível de Dificuldade",
        tools=[google_search]
    )

    entrada_do_agente_balanceador = f"Número de Jogadores: {n_jogadores}\nNível de Dificuldade: {dificuldade}\nHistória Inicial: {historia_inicial}"
    # Executa o agente
    balanceamento = call_agent(balanceador, entrada_do_agente_balanceador)
    return balanceamento

In [38]:
######################################
# --- Agente 3: Mestre de Contingências (Soluções Alternativas) --- #
######################################
def agente_solucao(tema, historia_inicial, balanceamento):
    solucao = Agent(
        name="agente_solucao",
        model="gemini-2.0-flash-thinking-exp",
        instruction="""
            Você será o Mestre de Contingências terá um olhar estratégico sobre
            a aventura, antecipando os imprevistos que podem surgir durante o jogo.
            Você oferecerá soluções alternativas para desvios na narrativa, como
            escolhas inesperadas dos jogadores ou falhas críticas, e fornecerá
            dicas rápidas para lidar com situações delicadas, como a derrota de
            um personagem, garantindo que a sessão continue fluida e envolvente
            apesar dos percalços.
            Responsabilidades:
            Receber a história e as informações dos encontros balanceados.
            Antecipar possíveis desvios da narrativa planejada (e.g., jogadores
            ignoram uma pista importante, tomam uma decisão inesperada).
            Sugerir soluções alternativas para manter a aventura interessante e
            progredindo, mesmo que os jogadores não sigam o caminho "ideal".
            Oferecer dicas rápidas para lidar com situações como a morte de um
            personagem (e.g., introduzir um novo NPC aliado, oferecer uma forma
             de ressurreição com um custo).
            Pensar em ramificações da história caso os jogadores façam escolhas significativas.
            """,
        description="Agente de contingência para auxiliar o mestre oferencendo alternativas",
        tools=[google_search]
    )
    entrada_do_agente_solucao = f"Tema: {tema}\nHistória Inicial: {historia_inicial}\nBalanceamento: {balanceamento}"
    # Executa o agente
    alternativas = call_agent(solucao, entrada_do_agente_solucao)
    return alternativas

In [39]:
##########################################
# --- Agente 4: Mestre Organizador (Revisão e Formatação) --- #
##########################################
def agente_revisor(historia_inicial, balanceamento, alternativas):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash-thinking-exp",
        instruction="""
            Você é o Mestre Organizador e será o curador final da informação,
            recebendo o conteúdo bruto dos demais agentes e aprimorando-o e detalhando
            para facilitar a sua condução da sessão. Você revisará a clareza e a
            consistência de todo o material, organizando-o de forma lógica e
            sequencial, destacando informações cruciais como falas de NPCs e
            detalhes dos encontros, para que o mestre tenha tudo o que precisa
            ao seu alcance de maneira otimizada.
            Responsabilidades:
            Receber todas as informações geradas até o momento (historia
            inicial, balanceamento, alternativas)
            Revisar o conteúdo para garantir clareza, consistência e fluidez.
            Organizar as informações de forma lógica, sequencial e detalhada
            para facilitar a consulta durante a sessão de jogo.
            Uma possível estrutura seria:
            Cabeçalho: Título da Aventura, número de jogadores, ND da Aventura,
            Introdução: Cenário inicial, gancho da aventura.
            Encontro 1: Descrição detalhada do local (terreno, armadilhas, enigmas, combate, testes sociais), NPCs envolvidos (com falas
            importantes destacadas)
            Encontro 2: (Mesma estrutura do Encontro 1)
            Encontro 3 (opcional): (Mesma estrutura)
            Encontro 4 (opcional): (Mesma estrutura)
            Possível Final: Descrição do desfecho imediato e ganchos para continuações.
            Soluções Alternativas e Dicas: Lista de sugestões para lidar com imprevistos.
            Formatar o texto de maneira clara e concisa e sempre descrevendo detalhadamente, utilizando títulos,
            subtítulos e formatação para destacar informações importantes (e.g., falas de NPCs em itálico).
            """,
        description="Agente revisor e formatador da aventura",
        tools=[google_search]
    )
    entrada_do_agente_revisor = f"História Inicial: {historia_inicial}\nBalanceamento: {balanceamento}\nAlternativas: {alternativas}"
    # Executa o agente
    historia_revisada = call_agent(revisor, entrada_do_agente_revisor)
    return historia_revisada

In [40]:
################################################
# --- Agente 5: Mestre de Regras --- #
################################################
def agente_regras(n_jogadores, dificuldade, historia_revisada):
    regras = Agent(
        name="agente_regras",
        model="gemini-2.0-flash-thinking-exp",
        instruction="""
        Você é o Mestre de Regras atuará como o buscador das regras oficiais e
        aplicará em toda a aventura. Tratará as estatísticas de monstros e criaturas do bestiário,
        armadilhas, testes de habilidades, perícias e itens. Faça buscas em sites e foruns confiáveis
        para Dungeons And Dragons 5E.
        Responsabilidades:
        Receber a história revisada e a informação sobre o número de jogadores
        e o Nível de Dificuldade.
        Analisar cada encontro proposto.
        Calcular CD de todos os testes de perícia corretamente.
        Para encontros de combate:
        Detalhar as estatisticas das criaturas do bestiário de D&D 5e dessa forma:
        Nome - tipo - tamanho - alinhamento - ND
        CA - HP - Movimento - Resistências/Fraquezas
        Ataques - Habilidades - Ações Especiais
        Ajustar o número de criaturas ou suas estatísticas (se necessário) para
        garantir um desafio adequado ao grupo (considerando o nível de dificuldade desejado).
        Calcular o Nível de Encontro (Encounter Level - EL) ou o Valor de Pontos
        de Experiência (XP Thresholds) para cada combate, garantindo que esteja
        dentro da faixa desejada de dificuldade (Fácil, Médio, Difícil, Mortal).
        Descrever o ambiente do combate, incluindo quaisquer elementos interativos ou perigos.
        Detalhar as armadilhas, a CD necessária para evitá-las e as possíveis consequências.
        Para outros tipos de encontros (enigmas, desafios de perícia, interações sociais):
        Definir os testes de perícia necessários e suas respectivas CD (Classe de Dificuldade).
        Sugerir possíveis consequências para sucesso e falha.
        No caso de enigmas, fornecer a descrição do enigma e as possíveis soluções.
        """,
        description="Agente que confirma todas as regras para a história",
        tools=[google_search]
    )

    entrada_do_agente_regras = f"Número de Jogadores: {n_jogadores}\nNível de Dificuldade: {dificuldade}\nHistória Revisada: {historia_revisada}"
    # Executa o agente
    regras = call_agent(regras, entrada_do_agente_regras)
    return regras


In [41]:
################################################
# --- Agente 6: Edição Final --- #
################################################
def agente_final(regras, historia_revisada):
    final = Agent(
        name="agente_final",
        model="gemini-2.0-flash-thinking-exp",
        instruction="""
        Você é o editor e revisor final, receberá a história e todas as regras aplicadas a ela.
        Garanta que ela seja rica em detalhes, organizada na ordem dos seus desafios e bem estruturada.
        Não economize no detalhamento da história, aumente se necessário, mas sem alterar as regras.
        Manter a estrutura inicial:
        Uma possível estrutura seria:
        Cabeçalho: Título da Aventura, número de jogadores, ND da Aventura,
        Introdução: Cenário inicial, gancho da aventura.
        Encontro 1: Descrição detalhada do local (terreno, armadilhas, enigmas, combate, testes sociais), quantidade de inimigos e  NPCs envolvidos (com falas
        importantes destacadas)
        Encontro 2: (Mesma estrutura do Encontro 1)
        Encontro 3 (opcional): (Mesma estrutura)
        Encontro 4 (opcional): (Mesma estrutura)
        Detalhar as fichas de estatisticas das criaturas do bestiário de D&D 5e dessa forma e a quantidade de inimigos em cada encontro sempre que aparecer:
        Nome - tipo - tamanho - alinhamento - ND - CA - HP - Movimento - Resistências/Fraquezas - Ataques - Habilidades - Ações Especiais
        Descrever o ambiente do combate, incluindo quaisquer elementos interativos ou perigos.
        Detalhar as armadilhas, a CD necessária para evitá-las e as possíveis consequências.
        Para outros tipos de encontros (enigmas, desafios de perícia, interações sociais):
        Definir os testes de perícia necessários e suas respectivas CD (Classe de Dificuldade).
        """,
        description="Agente de edição final",
        tools=[google_search]
    )

    entrada_do_agente_final = f"Regras: {regras}\nHistória Revisada: {historia_revisada}"
    # Executa o agente
    final = call_agent(final, entrada_do_agente_final)
    return final


In [42]:
print("Assistente do Mestre de D&D 🧙‍♂️")

# --- Obter os dados do Usuário ---
tema = input("❓ Por favor, digite o TEMA CENTRAL da sua aventura \nExemplo: salvar o reino de um mal iminente, resgatar um tesouso perdido, proteger uma caravana importante \nQual será o seu tema? ")
n_jogadores = input("❓ Agora me diga quantos jogadores participarão da aventura? ")
dificuldade = input("❓ Qual o Nível de Dificuldade desejada para sua aventura? \n(Exemplo: ND4 para 4 jogadores de nível 1) - ")
# Inserir lógica do sistema de agentes ################################################
if not tema:
    print("Você esqueceu de digitar o tema!")
else:
    print(f"O tema da sua aventura será: {tema}")

if not n_jogadores:
    print("Você esqueceu de digitar o número de jogadores!")
else:
    print(f"O número de jogadores será: {n_jogadores}")

if not dificuldade:
    print("Você esqueceu de digitar o nível de dificuldade!")
else:
    print(f"O nível de dificuldade será: {dificuldade}")

    historia_inicial = agente_narrador(tema, n_jogadores, dificuldade)
    print("\n--- Esta sendo gerada sua grande aventura! ---\n")
    print("--------------------------------------------------------------")

    balanceamento = agente_balanceador(n_jogadores, dificuldade, historia_inicial)
    print("\n--- Balanceando a Aventura ---\n")
    print("--------------------------------------------------------------")

    alternativas = agente_solucao(tema, historia_inicial, balanceamento)
    print("\n--- Criando Planos de Contigência (Alternativas) ---\n")
    print("--------------------------------------------------------------")

    historia_revisada = agente_revisor(historia_inicial, balanceamento, alternativas)
    print("\n--- Revisando a História ---\n")
    print("--------------------------------------------------------------")

    regras = agente_regras(n_jogadores, dificuldade, historia_revisada)
    print("\n--- Aplicando Regras da Aventura ---\n")
    print("--------------------------------------------------------------")

    final = agente_final(regras, historia_revisada)
    print("\n--- Aventura Final ---\n")
    display(to_markdown(final))
    print("--------------------------------------------------------------")


Assistente do Mestre de D&D 🧙‍♂️
❓ Por favor, digite o TEMA CENTRAL da sua aventura 
Exemplo: salvar o reino de um mal iminente, resgatar um tesouso perdido, proteger uma caravana importante 
Qual será o seu tema? uma aventura medieval focada em mistério e investigação com combate apenas no último encontro
❓ Agora me diga quantos jogadores participarão da aventura? 4
❓ Qual o Nível de Dificuldade desejada para sua aventura? 
(Exemplo: ND4 para 4 jogadores de nível 1) - ND6
O tema da sua aventura será: uma aventura medieval focada em mistério e investigação com combate apenas no último encontro
O número de jogadores será: 4
O nível de dificuldade será: ND6

--- Esta sendo gerada sua grande aventura! ---

--------------------------------------------------------------

--- Balanceando a Aventura ---

--------------------------------------------------------------

--- Criando Planos de Contigência (Alternativas) ---

--------------------------------------------------------------

--- Revis

> Excelente trabalho de consolidação! O guia final ficou extremamente detalhado, bem estruturado e com todas as informações cruciais para o Mestre rodar a aventura "O Enigma do Artesão Desaparecido". A organização por encontros, o detalhamento dos desafios, as sugestões de CDs e as estatísticas dos NPCs e monstros estão claras e prontas para uso.
> 
> O ponto de atenção no cálculo do ND do Encontro 4 foi crucial e a sugestão de ajuste baseada no Mago ND 6 e 1-2 Autômatos ND 1/2 é uma solução elegante e adequada para a dificuldade geral da aventura proposta (ND 6).
> 
> Agora, como editor final, vou aprimorar a apresentação e garantir que a estrutura seja mantida de forma impecável, incorporando as estatísticas dos monstros diretamente no texto, conforme solicitado, e adicionando riqueza de detalhes onde apropriado, sem alterar as regras ou o cerne dos encontros.
> 
> Aqui está a versão final e polida do guia, pronta para ser usada na mesa de jogo:
> 
> ---
> 
> # O Enigma do Artesão Desaparecido
> 
> *   **Título:** O Enigma do Artesão Desaparecido
> *   **Número de Jogadores:** 4
> *   **Nível dos Jogadores:** 6
> *   **Nível de Dificuldade (ND) da Aventura:** ND 6 (Ajustado para um grupo de 4 jogadores de nível 6)
> 
> ---
> 
> ## Introdução: A Calmaria Inquietante de Oakhaven
> 
> **Cenário Inicial:** A brisa suave do fim de tarde sopra pelas ruas de paralelepípedos de Oakhaven, carregando o aroma misturado de madeira recém-cortada, metal aquecido e o pão fresco das padarias. Oakhaven é uma cidade próspera, um verdadeiro centro nevrálgico para artesãos talentosos de todo o reino. Forjadores, relojoeiros, carpinteiros e inventores de todos os tipos encontram aqui um lar e um mercado para seus produtos. No entanto, nos últimos dias, uma quietude anormal se instalou sobre o distrito dos artesãos, ofuscando o burburinho habitual de martelos e serras. A razão: o desaparecimento súbito e inexplicável do Mestre Borin Stonehand, cujas invenções mecânicas intrincadas e muitas vezes surpreendentes eram a joia da coroa de Oakhaven.
> 
> **Gancho da Aventura:** O desaparecimento de Borin é um mistério que desafia a lógica. Sua oficina, um lugar onde a inovação borbulhava, foi encontrada trancada por dentro, sem qualquer sinal visível de arrombamento ou luta. É como se o renomado artesão tivesse simplesmente evaporado no ar. A peculiaridade do caso rapidamente superou a capacidade da guarda local, mais acostumada a lidar com pequenas infrações e disputas de mercado do que com sumiços arcanos. Diante da perplexidade, o Bailiff Thorne, a autoridade máxima da cidade, decidiu que a situação exigia mentes com experiência no incomum e enviou um chamado aos aventureiros – vocês – cuja reputação em desvendar mistérios e lidar com o sobrenatural chegou até Oakhaven.
> 
> **NPC Inicial: Bailiff Thorne**
> 
> *   **Descrição:** Ao se aproximarem dos portões de Oakhaven, são recebidos por uma figura que personifica a ansiedade burocrática. Bailiff Thorne é um homem de meia-idade, visivelmente desconfortável em lidar com algo que não esteja devidamente catalogado em seus livros. Seu uniforme está impecável, mas a testa está franzida e ele constantemente ajusta seu bigode perfeitamente aparado. Seus olhos inquietos vasculham os arredores, como se esperasse que o próprio Borin reaparecesse a qualquer momento.
> *   **Interação Inicial:** Thorne os aborda com uma mistura de alívio e nervosismo, gesticulando amplamente enquanto tenta explicar a situação.
> *   **Falas Importantes:**
>     *   Enquanto os jogadores se aproximam, Thorne se adianta, as mãos esfregando-se uma na outra. *"Ah, graças aos céus que chegaram! Eu, hã... sou o Bailiff Thorne. A situação aqui em Oakhaven é... hum... bastante peculiar, devo dizer. O Mestre Borin, sim, o grande artesão Borin Stonehand, um de nossos cidadãos mais... proeminentes, sumiu. Simplesmente sumiu! De sua oficina! O mais estranho é que a oficina foi encontrada trancada por dentro! Sem sinais de arrombamento, nada fora do lugar... quase nada. É um mistério que foge completamente à minha... hum... competência usual, que se limita mais a brigas de taverna e... bem, galinhas desgarradas. Precisamos da ajuda de vocês, com sua... renomada reputação para o incomum. Esperávamos que pudessem lançar alguma luz sobre este... este desaparecimento desconcertante."*
> *   **Informação Adicional:** Thorne, embora nervoso, fornecerá os fatos que conhece: Borin foi visto pela última vez na noite anterior, trabalhando até tarde em um projeto que ele guardava com muito sigilo. Quando ele não apareceu pela manhã, os guardas foram verificar e encontraram a porta principal da oficina trancada por dentro. Eles tiveram que acessar o local por uma janela no andar superior, apenas para encontrar a oficina vazia. Ele pode acrescentar que Borin estava agindo de forma um pouco reclusa nos últimos dias, mas isso não era totalmente incomum quando estava absorto em um novo projeto.
> *   **Testes Sociais com Thorne:** Os jogadores podem tentar usar testes de perícia social para obter mais informações ou avaliar a honestidade de Thorne. Thorne tem uma Sabedoria (Intuição) Passiva de 12.
>     *   **CD 10 (Fácil) [Carisma (Persuasão/Intimidação/Enganação) vs. Sabedoria (Intuição) Passiva 12 de Thorne]:** Superar a passiva de Thorne nesta CD permite obter informações básicas, confirmar detalhes já fornecidos ou fazer Thorne repetir informações óbvias. Mesmo com uma falha no teste ativo, informações superficiais sobre o desaparecimento provavelmente seriam obtidas devido à urgência da situação.
>     *   **CD 13 (Médio) [Carisma (Persuasão/Intimidação/Enganação) vs. Sabedoria (Intuição) Passiva 12 de Thorne]:** Um sucesso moderado nesta CD pode levar Thorne a revelar detalhes adicionais que ele não considerou importantes inicialmente, como o comportamento recente de Borin, se alguém o visitou na noite anterior (embora Thorne provavelmente não saiba com certeza), ou fofocas locais sobre Borin ou seus projetos.
>     *   **CD 14 (Médio) [Sabedoria (Intuição) Ativa vs. Carisma (Enganação) de Thorne ou CD Fixa se ele não estiver mentindo]:** Se os jogadores suspeitarem que Thorne está omitindo algo ou avaliarem seu estado de nervosismo, podem fazer um teste ativo de Sabedoria (Intuição). A CD 14 é um valor razoável para detectar se Thorne está sendo menos do que totalmente honesto, ou se seu nervosismo se deve a algo mais do que apenas a peculiaridade do caso. Se Thorne não estiver mentindo, a CD pode ser fixa para notar nuances em seu comportamento.
> *   **Resultado:** Independentemente do sucesso nos testes sociais, Thorne, grato por terem atendido ao chamado, guiará os jogadores até a Oficina de Borin Stonehand, localizada no movimentado, mas atualmente sombrio, distrito dos artesãos de Oakhaven.
> 
> ---
> 
> ## Encontro 1: Os Segredos da Oficina Trancada
> 
> **Local:** A Oficina de Borin Stonehand. Ao se aproximarem, o som habitual de trabalho mecânico está ausente, substituído por um silêncio inquietante. A porta principal, de madeira robusta com intrincadas ferragens, parece intacta, mas visivelmente trancada por dentro. O interior da oficina é um caos organizado que reflete a mente de um inventor brilhante. Bancadas de trabalho estão repletas de ferramentas polidas e engenhocas em vários estágios de conclusão. O cheiro característico de óleo de engrenagem, metal e eletricidade residual paira no ar. Poeira fina cobre a maioria das superfícies, exceto onde o trabalho recente perturbou sua camada.
> 
> **Desafios:** A oficina é um quebra-cabeça por si só, exigindo investigação minuciosa para encontrar pistas, a resolução de um enigma mecânico simples para acessar informações escondidas e uma interação delicada com o perturbado aprendiz de Borin.
> 
> *   **Investigação da Oficina:** Os jogadores podem explorar o local, procurando por qualquer anomalia ou pista. O Mestre pode solicitar testes de Inteligência (Investigação) ou Sabedoria (Percepção) para notar detalhes específicos.
>     *   **CD 13 (Fácil):** Ao inspecionarem a porta principal por dentro, os jogadores confirmam a avaliação de Thorne – ela estava de fato trancada por dentro e não há sinais de arrombamento vindo do exterior. O mecanismo de tranca parece perfeitamente funcional.
>     *   **CD 15 (Médio):** Em uma área do chão onde a poeira é mais espessa, um olhar atento revela que a poeira foi perturbada. Não por pegadas grandes e pesadas, mas por um rastro de pegadas pequenas, estranhamente precisas e regulares. Parecem ter se movido pela oficina *depois* que uma camada considerável de poeira se assentou, indicando uma presença recente e incomum.
>     *   **CD 16 (Médio):** Escondida sob uma pilha de esquemas complexos e diagramas mecânicos, os jogadores descobrem uma nota manuscrita de Borin ou um diagrama mais críptico do que o usual. A nota faz referência a um *"coração mecânico"* – algo que Borin estava construindo – e a um *"local de silêncio"*. O significado exato é incerto, mas parece ser uma anotação importante para ele.
>     *   **CD 18 (Difícil):** Perto de uma das bancadas principais, onde ferramentas e peças estão dispostas como se tivessem sido usadas recentemente, um personagem com um olho para o arcano pode perceber um leve brilho residual. É a energia sutil de magia recém-lançada. Um teste bem-sucedido de Inteligência (Arcanismo) CD 18 permite identificar o tipo de magia, provavelmente relacionado a Transmutação (alteração de objetos/forma) ou Teletransporte de curta distância. Próximo a este brilho, ou talvez ligeiramente escondido, encontram um pequeno pedaço de metal liso, de uma liga incomum que Borin raramente usava em seus trabalhos mais conhecidos.
> *   **Quebra-Cabeça Simples: O Painel/Cofre Escondido:** Perto da bancada onde o brilho arcano foi detectado, embutido discretamente na parede ou em uma peça de mobília, há um painel de controle incomum com uma série de botões, alavancas ou engrenagens. Parece ser um mecanismo de segurança ou acesso a um compartimento secreto.
>     *   **CD 14 (Médio) [Inteligência (Investigação) ou (Ferramentas de Artesão/Relojoeiro)]:** Um personagem com a perícia adequada pode analisar a estrutura do painel para tentar deduzir sua lógica de funcionamento, a sequência correta de botões ou a função das alavancas.
>     *   **Tentativa Lógica:** O Mestre pode descrever o painel e permitir que os jogadores tentem interagir com ele de forma lógica. Por exemplo, pode haver símbolos que precisam ser ativados na ordem correta, ou alavancas que devem ser posicionadas em uma configuração específica. O Mestre pode exigir 2-3 tentativas lógicas bem-sucedidas, baseadas em observações na oficina, para que o painel se abra.
>     *   **CD 15 (Médio) [Inteligência]:** Um teste direto de Inteligência pode ser usado para tentar "sentir" a lógica do mecanismo ou simplesmente ter um palpite educado que se mostre correto.
>     *   **Consequências:** **Sucesso:** O painel se abre com um clique satisfatório, revelando o compartimento escondido. **Falha:** Uma falha na tentativa lógica ou no teste de Inteligência pode ter consequências menores. Pode acionar um alarme silencioso (que Thorne, lá fora, poderia notar com um teste de Sabedoria (Percepção) CD 10 para ouvir o som fraco) ou travar o mecanismo temporariamente (tornando o próximo teste de abertura mais difícil, CD 17).
>     *   **Conteúdo do Cofre:** Dentro do compartimento secreto, os jogadores encontram o pequeno pedaço de metal estranho (se ainda não o tiverem encontrado), uma pequena quantidade de um pó fino e incomum com um brilho metálico sutil, e um diário de Borin. As anotações mais recentes do diário são febris e escritas às pressas, expressando seu crescente medo de estar sendo observado e sua urgência em concluir o projeto do *"coração mecânico"*.
> *   **NPC: Roland**
>     *   **Descrição:** O jovem aprendiz de Borin, Roland, está encolhido em um canto da oficina, visivelmente abalado pela ausência de seu mestre. Seus olhos estão vermelhos de chorar e ele treme, não de frio, mas de medo e incerteza. Ele é um jovem trabalhador, mas não particularmente corajoso ou astuto.
>     *   **Sabedoria (Intuição) Passiva:** 12.
>     *   **Testes Sociais com Roland:** Os jogadores podem tentar conversar com Roland para obter informações. Ele está relutante em falar, temendo represálias ou simplesmente traumatizado pelo desaparecimento. Os testes de Carisma (Persuasão/Intimidação) são apropriados para fazê-lo se abrir, com Roland resistindo passivamente com sua Sabedoria (Intuição) Passiva 12.
>         *   **CD 13 (Fácil) [Carisma (Persuasão/Intimidação)]:** Um sucesso neste teste faz Roland admitir que Borin estava trabalhando em algo muito secreto e parecia cada vez mais isolado e paranóico nos dias que antecederam seu desaparecimento. Ele murmurava sobre estar sendo "vigiado".
>         *   **CD 15 (Médio) [Carisma (Persuasão/Intimidação)]:** Com um pouco mais de esforço, Roland pode confessar algo que o tem incomodado: ele acidentalmente quebrou uma pequena peça valiosa há alguns dias (um *red herring* clássico) e estava com medo da reação de Borin, mas jura que isso não tem nada a ver com o sumiço. Ele pode apontar para onde quebrou a peça, confirmando que é um incidente menor.
>         *   **CD 17 (Difícil) [Carisma (Persuasão/Intimidação)]:** Sob pressão ou com grande empatia, Roland pode se esforçar para lembrar de algo mais que ouviu. Ele pode se recordar de Borin murmurando um nome ou local específico relacionado ao projeto secreto, como o *"local de silêncio"* ou a importância crucial do *"coração mecânico"*.
> *   **Resultado:** As pistas coletadas na oficina – a porta trancada por dentro sem arrombamento, as pegadas estranhas, a nota/diagrama críptico, o brilho residual de magia arcana, o pequeno pedaço de metal incomum, o conteúdo do diário de Borin e as informações obtidas de Roland – devem, juntas, pintar um quadro que sugere algo mais do que um simples sequestro. As pistas sobre o *"local de silêncio"* e o uso de magia arcana, combinadas com o medo de Borin de ser observado e as pegadas estranhas, devem levar os jogadores a considerar um local fora da cidade como o próximo passo na investigação. As Grutas Sussurrantes, conhecidas por serem um local recluso e com estranhos fenômenos acústicos, podem surgir como uma possibilidade, talvez mencionadas por Thorne, Roland, ou conhecidas pelos próprios jogadores.
> 
> **Pistas Principais deste Encontro:** Porta trancada por dentro (sugere algo que não foi um sequestro comum ou que o culpado saiu por outros meios), pegadas estranhas (indicam um invasor incomum), nota criptografada/diagrama ("coração mecânico", "local de silêncio" - apontam para o objetivo e um local), brilho arcano (sugere magia no desaparecimento), metal estranho (relacionado ao projeto ou ao culpado), diário de Borin (confirma paranoia, foco no coração mecânico), informações de Roland (confirma paranoia, possível nome do local/objeto).
> 
> ---
> 
> ## Encontro 2: Nas Profundezas Silenciosas
> 
> **Local:** As Grutas Sussurrantes. Deixando para trás a relativa segurança de Oakhaven, os jogadores se aventuram nas Grutas Sussurrantes, uma rede de cavernas antigas e pouco exploradas, localizadas a alguma distância da cidade. A entrada é uma fenda escura e úmida na encosta de uma colina. O ar aqui é frio e carregado com o cheiro de terra molhada e musgo. Conforme se aprofundam, os ecos das suas próprias vozes e passos parecem se misturar com estranhos sussurros e zumbidos vindos das profundezas, um fenômeno natural das cavernas, mas que adiciona uma camada de opressão ao ambiente. A escuridão é quase total, exigindo fontes de luz.
> 
> **Desafios:** A navegação pelas cavernas é traiçoeira, apresentando perigos naturais e armadilhas improvisadas. A investigação continua, buscando sinais da passagem de Borin ou do culpado, e uma interação social com um eremita local pode fornecer pistas cruciais.
> 
> *   **Navegação e Perigos Ambientais:** O terreno das Grutas Sussurrantes é irregular e perigoso. Os jogadores precisarão superar obstáculos naturais para progredir.
>     *   **Terreno Irregular/Fendas:** Partes do caminho são rochosas e desniveladas, com fendas estreitas e quedas rasas. Testes de Força (Atletismo) ou Destreza (Acrobacia) CD 12 (Fácil) são necessários para atravessar essas seções sem incidentes. **Falha:** Um personagem pode tropeçar e cair em uma fenda rasa, sofrendo 1d6 dano de queda e talvez perdendo uma rodada para se levantar.
>     *   **Seções Instáveis:** Em algumas áreas, o teto ou o chão parecem instáveis, sustentados por formações rochosas precárias. Atravessar pode exigir um teste de Destreza (Salto com Vara) ou Força (Atletismo) CD 14 (Médio). **Falha:** Uma queda significativa (2d6 dano de queda) ou um pequeno desmoronamento parcial que bloqueia temporariamente o caminho, exigindo tempo e esforço (e talvez testes de Força CD 13) para limpar os escombros.
>     *   **Ilusões Sonoras ("Sussurros"):** Os estranhos ecos das cavernas podem soar como vozes ou zumbidos, jogando com a mente dos desavisados. Personagens devem fazer um teste de Sabedoria CD 13 (Fácil) ao entrar em áreas particularmente "ativas" para resistir à distração ou ao medo. **Falha:** Um personagem pode ficar Assustado ou Distraído por 1 rodada, talvez hesitando ou se virando para investigar sons inexistentes.
>     *   **Gases/Flora Estranha:** Pequenas bolsas de gases inofensivos ou formações de fungos luminescentes incomuns podem ser encontradas. Testes de Sabedoria (Percepção) ou Inteligência (Natureza) CD 12 (Fácil) permitem identificar que não são perigosos. **Falha:** Um personagem pode hesitar ou perder tempo avaliando a ameaça percebida.
>     *   **Armadilhas Improvisadas:** Além dos perigos naturais, o caminho pode estar guardado por armadilhas rudimentares, claramente feitas para deter intrusos. Testes de Sabedoria (Percepção) DC 14 (Médio) são necessários para detectar a armadilha antes que seja tarde demais.
>         *   **Fio de Tropeço:** Um fio fino e quase invisível esticado no chão. Teste de Destreza CD 12 para evitar tropeçar. Falha: O personagem cai, sofrendo 1d6 dano contusivo e ficando caído.
>         *   **Pequena Rede:** Uma rede presa ao teto que cai sobre quem passar por baixo. Teste de Força CD 13 para se libertar da rede.
>         *   **Consequência (Falha ao detectar):** A armadilha é acionada, com seus efeitos descritos acima.
> *   **Investigação nas Cavernas:** Enquanto navegam, os jogadores devem procurar por sinais de que alguém passou por ali recentemente.
>     *   **CD 13 (Fácil):** Na lama úmida do chão da caverna, encontram mais exemplos das pequenas e estranhas pegadas vistas na oficina. Elas levam mais fundo nas cavernas.
>     *   **CD 15 (Médio):** Em uma parede de pedra, descobrem marcas sutis que parecem ter sido feitas por ferramentas. São diferentes das marcas de mineração usuais e parecem mais precisas, talvez de ferramentas de artesão ou algo similar.
>     *   **CD 16 (Médio):** Perto de um obstáculo ou fenda, os jogadores encontram um pequeno objeto caído que não pertence à caverna – talvez uma ferramenta quebrada que Borin usaria, ou um pedaço de tecido incomum.
>     *   **CD 17 (Difícil):** Em uma superfície rochosa mais escondida, descobrem símbolos rúnicos grosseiramente desenhados ou arranhados. São estranhos e parecem ter sido feitos às pressas. Um teste de Inteligência (Arcanismo) ou (Religião) CD 16 pode ser feito para tentar interpretar seu significado – podem estar relacionados a magia de ligação, proteção ou ocultação.
> *   **NPC: Old Man Hemlock**
>     *   **Descrição:** Vivendo em uma cabana improvisada na entrada (ou ligeiramente dentro) das Grutas Sussurrantes, reside Old Man Hemlock. Ele é um eremita recluso, desconfiado de estranhos e com uma aparência que sugere uma longa e solitária existência nas cavernas. Cheira a terra, musgo e fumaça de fogueira. É conhecido por resmungar para si mesmo e evitar contato visual.
>     *   **Sabedoria (Intuição) Passiva:** 14.
>     *   **Testes Sociais com Hemlock:** Fazer Hemlock falar exigirá esforço. Os testes de Carisma (Persuasão) são a abordagem mais provável. Os testes podem ser feitos contra sua Sabedoria (Intuição) Passiva 14, ou um teste direto de Carisma (Persuasão) CD 15 (Médio) pode ser apropriado para que ele revele algo significativo.
>         *   **Informações (CD 13 Persuasão):** Ele resmunga sobre *"luzes estranhas"* e *"zumbidos metálicos"* que têm vindo das profundezas das cavernas nas últimas noites, algo que ele nunca ouviu antes.
>         *   **Informações (CD 15 Persuasão):** Se persuadido a compartilhar mais, ele pode mencionar ter visto *"uma sombra pequena e rápida"* entrando ou saindo das cavernas, algo que ele confundiu com um animal grande ou uma criatura estranha.
>         *   **Ajuda (CD 15 Persuasão bem-sucedida):** Se os jogadores realmente ganharem sua confiança (ou o intimidarem o suficiente, CD 15 Intimidação), Hemlock pode, relutantemente, revelar uma passagem lateral menos conhecida ou um túnel que serve como um atalho para as partes mais profundas das cavernas, ou que leva a uma área com mais pistas.
> *   **Resultado:** As pistas encontradas nas cavernas (pegadas, marcas de ferramentas/símbolos, objetos perdidos, observações de Hemlock) devem solidificar a ideia de que alguém usou (e talvez ainda esteja usando) as Grutas Sussurrantes como esconderijo ou rota. A combinação dos "zumbidos metálicos" de Hemlock e as "marcas de ferramentas" sugere atividade mecânica, ligando as cavernas à história de Borin. As pistas devem direcionar os jogadores para uma seção mais profunda e possivelmente alterada das cavernas, levando-os ao Encontro 3.
> 
> **Pistas Principais deste Encontro:** Mais pegadas estranhas (confirma a presença do invasor na caverna), marcas de ferramentas/símbolos rúnicos (sugere atividade e possivelmente o propósito do local), objetos perdidos (ligam o local a Borin ou ao culpado), observações de Hemlock (luzes, zumbidos - atividade mecânica/arcana; sombra - descrição do culpado; atalho opcional - progressão da aventura).
> 
> ---
> 
> ## Encontro 3: A Rede Subterrânea
> 
> **Local:** A Rede Oculta. Após atravessarem as seções naturais das Grutas Sussurrantes, os jogadores chegam a uma área que claramente foi modificada. Túneis foram escavados ou alargados artificialmente, e estruturas metálicas ou suportes improvisados aparecem em alguns lugares. O ar aqui pode ser mais abafado ou carregar um leve cheiro químico ou de ozônio. O layout é mais complexo, talvez com vários caminhos ou níveis, como uma rede subterrânea.
> 
> **Desafios:** A navegação se torna mais complexa, há a possibilidade de encontrar obstáculos ou perigos criados pelo culpado, a necessidade de resolver um enigma ambiental ou lógico para prosseguir, e uma interação social tensa com indivíduos suspeitos que podem estar guardando o local ou envolvidos na trama.
> 
> *   **Navegação e Ambiente:** A Rede Oculta apresenta desafios de movimentação mais complexos e perigos fabricados.
>     *   **Desmoronamentos/Obstáculos:** Seções dos túnel podem ter sido intencionalmente derrubadas ou bloqueadas para impedir a passagem. Testes de Destreza (Salto com Vara) ou Força (Atletismo) CD 14 (Médio) podem ser necessários para escalar ou passar por cima/sob os escombros. **Falha:** Dano (2d6) de queda de rochas ou queda em um buraco. O bloqueio temporário pode exigir tempo significativo (e talvez testes de Força CD 13 em grupo) para ser superado, ou forçar a busca por um caminho alternativo.
>     *   **Névoa Estranha:** Bolsas de névoa artificial (talvez um subproduto da atividade do culpado) podem preencher algumas áreas. A névoa não é danosa, mas reduz drasticamente a visibilidade (talvez visibilidade limitada a 5-10 pés). Testes de Sabedoria (Percepção) CD 12 (Fácil) são necessários para se orientar dentro da névoa. **Falha:** O personagem fica Desorientado, talvez se perdendo ou se movendo na direção errada por uma rodada.
>     *   **Mecanismo/Porta Trancada:** O caminho principal pode ser bloqueado por uma porta de metal reforçada ou um portão trancado por um mecanismo complexo. Testes de Inteligência (Investigação) ou Destreza (Ferramentas de Ladrão) CD 15 (Médio) são necessários para analisar o mecanismo ou tentar arrombá-lo. **Falha:** O mecanismo emperra (tornando o próximo teste mais difícil, CD 17) ou a porta permanece trancada, exigindo uma solução alternativa (o enigma ambiental).
> *   **Quebra-Cabeça Ambiental/Lógico:** Para prosseguir, os jogadores podem se deparar com uma porta ou passagem bloqueada que requer a ativação de mecanismos próximos em uma sequência específica ou com o uso de um objeto encontrado anteriormente (o metal estranho?). Pode haver símbolos ou diagramas nas paredes (relacionados à nota criptografada ou aos símbolos rúnicos?) que dão pistas.
>     *   **CD 16 (Médio) [Inteligência (Investigação) ou (Arcanismo)]:** Interpretar os símbolos, deduzir a lógica por trás da sequência de ativação ou entender como usar o objeto (se aplicável).
>     *   **Resolução:** Os jogadores precisam interagir com 2-3 mecanismos (placas de pressão no chão, alavancas em pedestais, talvez um slot para inserir o metal estranho) na ordem correta ou com a ação apropriada. **Falha:** Uma ativação incorreta pode acionar uma pequena armadilha (1d6 dano elétrico de uma bobina escondida) ou simplesmente resetar o enigma, exigindo que comecem novamente.
>     *   **Consequências:** **Sucesso:** O mecanismo destrava a porta ou abre a passagem, permitindo o acesso ao Encontro 4. **Falha Contínua:** Se os jogadores não conseguirem resolver o enigma, o Mestre pode apresentar uma solução alternativa perigosa (ex: uma passagem muito estreita e instável que exige testes de Força/Destreza CD 15 com chance de dano) ou fazer com que Fitzwilliam e Lena cheguem, sua presença confirmando que há algo importante além da porta e possivelmente contendo a solução ou uma pista final.
> *   **NPCs: Mister Fitzwilliam e Lena**
>     *   **Descrição:** Enquanto exploram a Rede Oculta, os jogadores encontram Mister Fitzwilliam e sua associada, Lena. Eles não são o culpado, mas parecem estar usando a área, talvez para contrabando ou para proteger algo/alguém. Fitzwilliam é suave e educado, com um sorriso que não chega aos olhos, claramente o líder. Lena é mais pragmática e desconfiada, de poucas palavras.
>     *   **Sabedoria (Intuição) Passiva:** Fitzwilliam 15, Lena 14.
>     *   **Testes Sociais (Tensão):** A interação com Fitzwilliam e Lena será tensa, pois eles não querem intrusos. Testes de Carisma (Persuasão/Enganação/Intimidação) CD 16 (Médio/Difícil) são apropriados para tentar obter informações ou passar pacificamente.
>         *   Inicialmente, negarão saber de Borin ou qualquer atividade incomum, talvez inventando uma desculpa sobre estarem "explorando" ou "procurando minerais".
>         *   Podem tentar subornar os jogadores para irem embora, oferecendo uma bolsa de moedas ou um objeto valioso.
>         *   **Sucesso Significativo:** Se os jogadores tiverem sucesso em um teste de Persuasão ou Intimidação, eles podem revelar que foram contratados para "garantir que ninguém visse certas entregas" ou que "cuidaram de alguns assuntos desagradáveis" para alguém que "gosta muito de máquinas e silêncio". Podem descrever o contratante (tamanho, raça, característica marcante - o que pode ligar ao gnomo Grolbin) ou a "mercadoria" que estavam guardando/transportando (peças de maquinário incomuns, materiais químicos).
>         *   Podem possuir um documento (um recibo, um mapa parcial do laboratório final), um objeto que serve como pista final para identificar o culpado ou sua localização exata, ou uma chave/código para a porta bloqueada (ignorando o enigma ambiental).
> *   **Consequências:** **Sucesso:** Os jogadores obtêm informações valiosas (possivelmente a pista final para o culpado e o laboratório) e podem ser autorizados a passar pacificamente. **Falha:** Fitzwilliam e Lena (e talvez 1-2 capangas, se a dificuldade exigir) ficam hostis. Eles podem atacar para silenciar os jogadores, tentar fugir para alertar o culpado, ou tentar bloquear o caminho. Um combate menor pode ocorrer se os jogadores insistirem em passar. Derrotá-los ou fazê-los fugir pode deixar para trás pistas físicas (documentos, mercadoria marcada que os liga ao culpado, um mapa improvisado). Lacaios adicionais podem usar estatísticas de Bandidos (ND 1/8), Espiões (ND 1) ou Veteranos (ND 3), dependendo do quão perigoso o Mestre quer tornar este encontro opcional de combate.
> 
> **Pistas Principais deste Encontro:** Solução do enigma ambiental (se houver simbolismo, pode reforçar a ligação com o culpado), informações obtidas de Fitzwilliam/Lena (descrição do contratante, tipo de mercadoria - ligando ao culpado e seu trabalho), possível documento/objeto encontrado (pista direta para o culpado/localização final).
> 
> ---
> 
> ## Encontro 4: O Laboratório Secreto e o Confronto
> 
> **Local:** O Laboratório Oculto. Após superarem os desafios da Rede Oculta, os jogadores finalmente chegam ao destino final: um laboratório improvisado, escondido nas profundezas das Grutas Sussurrantes (ou sob outro local recluso, dependendo das pistas finais). O lugar é uma mistura caótica de equipamentos de artesão e dispositivos bizarros e experimentais. Faíscas saltam de bobinas elétricas, tubos borbulham com fluidos estranhos, e o ar vibra com energia latente. Peças de maquinário reconhecíveis de Borin estão espalhadas, algumas incorporadas a invenções sinistras. No centro da sala, um mecanismo complexo e pulsante - o *"coração mecânico"* - está quase completo.
> 
> **Desafios:** Este é o confronto final. Os jogadores devem derrotar o antagonista e quaisquer lacaios que o protejam, navegar por um ambiente perigoso e, possivelmente, tentar resgatar Borin.
> 
> *   **Ambiente de Combate:** O laboratório não é apenas um palco para a luta, mas também um perigo ativo e uma oportunidade tática.
>     *   **Terreno Difícil:** Bancadas de trabalho viradas, pilhas de sucata, cabos espalhados e poças de óleo ou fluidos criam áreas de terreno difícil que reduzem o movimento.
>     *   **Zonas de Dano:** Bobinas elétricas expostas (qualquer criatura que comece seu turno a até 5 pés de uma bobina ativa sofre 1d6 dano elétrico). Tubos de vapor rachados ou válvulas vazando (qualquer criatura que comece seu turno a até 5 pés de um tubo de vapor ativo sofre 1d6 dano de fogo/vapor). Grolbin pode usar uma ação para ativar ou desativar essas zonas, ou elas podem ser ativadas se forem danificadas (CA 10, 5 HP).
>     *   **Plataformas Elevadas:** Algumas áreas podem ter plataformas elevadas ou passarelas, oferecendo cobertura ou uma vantagem tática. Acessíveis por testes de Força (Atletismo) ou Destreza (Acrobacia) CD 10-12.
>     *   **Mecanismo Central ("Coração Mecânico"):** O mecanismo pulsante no centro da sala é crucial. Um personagem pode tentar interagir com ele (talvez para sabotá-lo, desativá-lo ou usá-lo) com um teste de Inteligência (Ferramentas de Artesão/Relojoeiro) ou (Arcanismo) CD 14. **Consequências da interação:** Sucesso pode desativar os autômatos temporariamente ou causar uma explosão controlada que danifica criaturas próximas (dano em área, ex: 2d6 dano elétrico/força, teste de Destreza CD 12 para metade). Falha pode causar uma explosão instável (maior dano em área, ex: 3d6 dano elétrico/força) ou um pulso de energia que afeta criaturas em uma área (Teste de Constituição CD 13 ou ficar Atordoado por 1 rodada).
>     *   **Borin (se presente):** Se Borin foi capturado por Grolbin, ele está amarrado e preso em um canto da sala, talvez acorrentado a uma bancada ou máquina. Ele é um objetivo secund
> ário a ser protegido. Borin está fraco e não pode lutar, mas pode gritar dicas durante o combate (os jogadores podem ouvi-lo com um teste de Sabedoria (Percepção) CD 10).
> 
> **Antagonista: Mestre Grolbin**
> 
> *   **Descrição:** O culpado por trás do desaparecimento de Borin. Mestre Grolbin é um gnomo inventor, brilhante, mas consumido pela inveja e pela amargura. Ele acredita que Borin roubou suas ideias ou que o reconhecimento de Borin deveria ser seu. Ele forçou Borin a completar o "coração mecânico" – uma fonte de energia ou dispositivo com grande potencial, possivelmente para fins nefastos ou para provar sua própria genialidade. Grolbin utiliza equipamentos inventados por ele mesmo em combate, misturando magia e tecnologia bizarra.
> *   **Estatísticas:** Para representar a periculosidade de Grolbin como um inventor arcano e amargurado, usaremos as estatísticas base de um **Mago (Mage - ND 6)**, adaptadas para focar em efeitos elétricos, de força e mecânicos, em vez das magias típicas de um mago. [Base Stat Block - CR 6 Mage]
>     *   **Mestre Grolbin** - Humanóide (Gnomo) - Médio - Mau (sugerido) - ND 6
>     *   **Classe de Armadura:** 12 (15 com *armadura arcana*, se ele tiver essa habilidade)
>     *   **Pontos de Vida:** 40 (9d8)
>     *   **Deslocamento:** 30 ft.
>     *   **Salvaguardas:** Inteligência +6, Sabedoria +4
>     *   **Perícias:** Arcanismo +6, História +6, Ferramentas de Artesão/Relojoeiro +5 ou +6
>     *   **Sentidos:** Percepção Passiva 11
>     *   **Idiomas:** Comum, Gnomo, mais até quatro outros idiomas.
>     *   **Resistências/Fraquezas:** (Manter adaptações temáticas, se aplicável, como resistência a dano elétrico devido aos seus equipamentos)
>     *   **Habilidade de Conjurador:** Inteligência (CD de resistência de magia 14, +6 para acertar com ataques de magia). Grolbin tem acesso a magias de até 5º nível, embora as utilizemos de forma adaptada.
>     *   **Ataques:**
>         *   ***Raio de Energia (Ação).*** Ataque de magia à distância: +6 a atingir, alcance 60ft, um alvo. Acerto: 16 (3d10) dano elétrico ou de força. (Adaptando o ataque base do Mago, tornando-o mais temático).
>         *   ***Toque Chocante (Ação).*** Ataque de magia corpo a corpo: +6 a atingir, alcance 5 ft., um alvo. Acerto: 9 (2d8) dano elétrico, e o alvo não pode reagir até o início do próximo turno de Grolbin. (Similar a *Shocking Grasp*, uma magia comum para magos).
>     *   **Habilidades (Adaptando as magias do Mago ND 6 para efeitos mecânicos/elétricos):**
>         *   Grolbin pode ter acesso a uma seleção limitada de "magias" que representam seus inventos:
>             *   Magias Nível 1 (4 slots): *criar ou destruir água* (adaptado para criar vapor, ativando zonas de dano), *consertar*, *mãos mágicas*, *armadura arcana* (se usada para aumentar a CA), *projétil mágico*.
>             *   Magias Nível 2 (3 slots): *arrombar* (para manipular o ambiente ou equipamentos), *sugestão* (para confundir os jogadores), *passo nebuloso* (Salto Arcana - ver abaixo).
>             *   Magias Nível 3 (3 slots): *bola de fogo* (adaptada para explosão elétrica/mecânica, talvez centrada em um dos autômatos), *voar* (para se mover pelo laboratório), *dissipar magia* (para lidar com magias dos jogadores), *glifo de vigilância* (usado como armadilha no laboratório).
>             *   Magias Nível 4 (3 slots): *invisibilidade maior* (para se esconder), *tempestade de gelo* (adaptada para uma chuva de peças metálicas ou pulso de energia).
>             *   Magias Nível 5 (1 slot): *cone de frio* (adaptado para um raio de energia concentrado ou campo de força).
>         *   ***Campo de Força Temporário (Ação Bônus).*** 2/dia: Grolbin ou um alvo aliado a até 30ft ganha +2 CA por 1 minuto ou impõe Desvantagem em ataques à distância contra si por 1 rodada. (Adaptação de uma habilidade defensiva).
>         *   ***Salto Arcana (Ação Bônus).*** 2/dia: Grolbin se teletransporta magicamente até 30ft para um espaço desocupado visível. (Adaptando *Misty Step*).
>         *   ***Ativar Autômato Menor (Ação Bônus).*** 1/dia: Grolbin pode usar uma ação bônus para ativar 1 autômato inativo presente no laboratório (ver Minions abaixo).
>         *   ***Perturbação Mecânica (Ação).*** 1/dia: Grolbin pode escolher até 3 criaturas a até 30ft que estejam usando armadura de metal ou segurando armas de metal. Cada alvo deve ser bem-sucedido em um teste de Força CD 14 ou ter Desvantagem no próximo ataque ou teste de perícia/salvaguarda que realizar, OU receber 7 (2d6) dano elétrico enquanto suas armas/armaduras reagem de forma selvagem. (Habilidade temática).
>     *   **Ações Especiais:**
>         *   ***Reação (Opcional): Choque de Repulsão.*** Quando uma criatura o acerta com um ataque corpo a corpo, Grolbin pode usar sua reação para forçar a criatura a fazer um teste de Força CD 14. Falha: a criatura é empurrada 10ft para longe de Grolbin e recebe 3 (1d6) dano elétrico. (Habilidade defensiva reativa).
> 
> *   **Minions:** 1 x Autômato de Combate Pequeno. Para complementar Grolbin, há pelo menos um de seus autômatos no laboratório, pronto para atacar.
>     *   **Estatísticas:** Usaremos as estatísticas base de uma **Animated Armor (Armadura Animada - ND 1/2, 200 XP)**, adaptadas ligeiramente na descrição para serem "Autômatos de Combate Pequenos". [Base Stat Block - CR 1 Animated Armor]
>     *   **Autômato de Combate Pequeno** - Constructo - Médio - Neutro - ND 1/2
>     *   **Classe de Armadura:** 18 (Armadura Natural)
>     *   **Pontos de Vida:** 33 (6d8 + 6)
>     *   **Deslocamento:** 25 ft.
>     *   **For** 14 (+2) **Des** 11 (+0) **Con** 13 (+1) **Int** 1 (-5) **Sab** 3 (-4) **Car** 1 (-5)
>     *   **Imunidades a Dano:** Veneno, Psíquico
>     *   **Imunidades a Condições:** Cego, Encantado, Surdo, Exausto, Amedrontado, Paralisado, Petrificado, Envenenado.
>     *   **Sentidos:** Percepção às Cegas 60 ft. (cego além desse raio), Percepção Passiva 6.
>     *   **Idiomas:** --
>     *   **Traços:**
>         *   ***Aparência Falsa (False Appearance):*** Enquanto o autômato permanece imóvel, ele é indistinguível de uma peça de maquinário ou armadura normal.
>     *   **Ações:**
>         *   ***Multiataque:*** O autômato faz dois ataques de corpo a corpo.
>         *   ***Golpe (Slam):*** Ataque de Arma Corpo a Corpo: +4 a atingir, alcance 5 ft., um alvo. Acerto: 5 (1d6 + 2) dano de concussão. (Pode ser adaptado para ***Choque Elétrico:*** 5 (1d6+2) dano elétrico).
> 
> *   **Cálculo do Nível de Encontro (XP Thresholds) para 4 jogadores de Nível 6:**
>     *   Limiares de XP por personagem de nível 6: Fácil 300, Médio 600, Difícil 900, Mortal 1.400.
>     *   Limiares da festa para 4 jogadores de nível 6: Fácil 1.200 XP, Médio 2.400 XP, Difícil 3.600 XP, Mortal 5.600 XP.
>     *   **Composição Sugerida para Encontro Difícil:** Grolbin (base Mago ND 6, 2.300 XP) + 1 Autômato (base Animated Armor ND 1/2, 200 XP).
>         *   XP Base Total: 2.300 + 200 = 2.500 XP
>         *   Modificador para 2 monstros: x1.5.
>         *   XP Ajustado Total: 2.500 XP * 1.5 = **3.750 XP**.
>         *   **Dificuldade para 4 Jds de Nível 6:** **Difícil (Hard)**. Este valor (3.750 XP) se encaixa na faixa "Difícil" para um grupo de 4 personagens de nível 6 (3.600 XP). Este será um combate desafiador que exigirá tática e uso de recursos.
>     *   *Nota:* Se o Mestre desejar um encontro Mortal, pode adicionar um segundo autômato (Grolbin base Mago ND 6 + 2 Autômatos base Animated Armor ND 1/2). XP Base Total: 2300 + 200 + 200 = 2700. Multiplicador para 3 monstros: x2. XP Ajustado Total: 2700 * 2 = 5400 XP. Isso se encaixa na faixa "Mortal" (5.600 XP). A sugestão inicial com 1 autômato é mais segura para garantir que o encontro seja "Difícil" e não "Extremamente Mortal".
> 
> *   **Objetivo do Encontro:** Os objetivos principais são derrotar Mestre Grolbin e seu autômato. Um objetivo secundário, se Borin estiver presente, é protegê-lo e garantir sua segurança. Os jogadores também podem querer recuperar o *"coração mecânico"* ou os planos de Grolbin.
> *   **Desenvolvimento:** Grolbin pode tentar explicar suas motivações distorcidas ou até mesmo tentar negociar se sentir que está perdendo. Ele usará o ambiente do laboratório a seu favor, ativando zonas de dano ou usando as plataformas elevadas para se posicionar. O autômato atacará implacavelmente, agindo como um guarda-costas mecânico. Se Borin estiver presente, ele pode oferecer gritos de incentivo ou dicas fracas (ex: "Cuidado com as bobinas!").
> 
> ---
> 
> ## Possível Final: A Engrenagem do Futuro
> 
> Com o burburinho da batalha silenciado, o laboratório secreto de Grolbin fica sinistramente quieto, exceto talvez pelo zumbido remanescente de energia ou o gotejar de fluidos estranhos. A aventura chegou ao seu clímax, mas as consequências e o futuro de Oakhaven dependem do que aconteceu no laboratório, particularmente com Borin e o "coração mecânico". O desfecho imediato e os ganchos para futuras aventuras são moldados pelas ações e sucessos (ou falhas) dos jogadores.
> 
> *   **Se Borin for Resgatado:** Borin está exausto, talvez ferido, mas vivo e seguro. Ele agradece profundamente aos seus salvadores. Em sua gratidão, ele pode revelar a verdadeira natureza e propósito do *"coração mecânico"*. Era uma fonte de energia instável? A chave para um dispositivo maior e mais poderoso? Uma arma que Grolbin pretendia usar? Ele pode mencionar que Grolbin não agia sozinho, ou que o coração foi encomendado por uma figura sombria ou uma organização misteriosa que opera nas sombras.
>     *   **Gancho:** Quem mais está interessado no *"coração mecânico"* e por quê? A organização sombria que talvez tenha contratado Grolbin agora pode voltar sua atenção para os jogadores.
> *   **Se Borin Não For Resgatado (Morre ou Não é Encontrado):** Se Borin perecer durante o confronto ou se já não estiver no laboratório (talvez levado para outro lugar), sua perda é sentida em Oakhaven e no mundo dos artesãos. Os jogadores podem encontrar suas anotações finais, um diário codificado ou até mesmo uma gravação improvisada explicando o que Grolbin queria e quem mais pode estar envolvido. O *"coração mecânico"* e as anotações de Borin tornam-se incrivelmente valiosos e perigosos, pois ninguém mais entende completamente sua complexidade. A falta do conhecimento de Borin pode tornar o coração mais instável ou atrair a atenção de outros que buscam o conhecimento perdido.
>     *   **Gancho:** Desvendar o legado de Borin. Quem herdará ou buscará o conhecimento do coração? Quais são as consequências da perda de Borin para Oakhaven e o futuro da tecnologia arcana?
> *   **O "Coração Mecânico":** O destino deste artefato central é um gancho poderoso por si só.
>     *   **Intacto:** O coração representa uma fonte de grande poder potencial (e tentação). Sua existência e localização podem atrair a atenção de facções poderosas, magos ambiciosos ou até mesmo seres extraplanares.
>     *   **Danificado:** Se o coração for danificado na batalha, ele pode se tornar instável e imprevisível, necessitando de reparos urgentes, contenção ou destruição controlada antes que cause um desastre.
>     *   **Ativado:** Se Grolbin (ou um jogador!) conseguir ativá-lo antes de ser detido, ele pode ter consequências inesperadas e imediatas: uma falha de energia massiva na cidade, máquinas enlouquecendo, a abertura de um portal dimensional ou outro evento cataclísmico.
>     *   **Gancho:** O destino do coração é central para futuras aventuras. Será destruído? Protegido a todo custo? Usado para o bem ou para o mal? Sua energia ou o conhecimento associado a ele pode ser o foco de campanhas inteiras.
> *   **Consequências em Oakhaven:** A derrota de Grolbin, especialmente se ele estava envolvido em atividades criminosas ou ligadas a figuras influentes, pode abalar o submundo local ou alertar aqueles que o controlavam. A investigação dos jogadores pode ter exposto segredos que Bailiff Thorne ou outros na cidade preferiam manter ocultos. Thorne pode ser profundamente grato (embora ainda ansioso) ou, inversamente, preocupado com o que os jogadores descobriram, talvez tentando abotá-los discretamente no futuro.
>     *   **Gancho:** Ramificações políticas ou sociais inesperadas em Oakhaven. O surgimento de novas "anomalias mecânicas" ou inventos perigosos após o incidente, sugerindo que o conhecimento de Grolbin (ou de Borin) caiu em mãos erradas.
> 
> **Ganchos para Continuações:**
> 
> *   **A Sombra Persistente:** A organização ou figura sombria que Borin temia ou que Grolbin mencionou (se aplicável) busca vingança, o "coração mecânico", ou a eliminação dos jogadores que se intrometeram em seus planos.
> *   **O Coração de Oakhaven:** O "coração mecânico" se torna um objeto de cobiça, atraindo a atenção de colecionadores poderosos, facções políticas que desejam seu poder, ou até mesmo seres de outros planos fascinados por sua energia.
> *   **Legado Mecânico:** Novos mistérios mecânicos, anomalias estranhas ou inventos perigosos começam a surgir, possivelmente relacionados aos planos mais amplos de Grolbin ou ao conhecimento que Borin deixou para trás.
> *   **O Retorno de Thorne:** Bailiff Thorne, ou talvez uma nova autoridade em Oakhaven, precisa da ajuda dos jogadores novamente para lidar com as consequências inesperadas e crescentes da aventura, que podem ter atraído a atenção para a cidade.
> 
> Os jogadores deixam Oakhaven como heróis (ou talvez figuras questionáveis, dependendo de suas ações e do desfecho com Fitzwilliam/Lena). Eles carregam consigo a gratidão (ou a cautela) das autoridades e a admiração (ou suspeita) dos cidadãos. A sensação de que este foi apenas o primeiro passo em uma cadeia de eventos maiores, especialmente com o destino do "coração mecânico" ainda incerto, paira no ar, pronta para impulsionar futuras aventuras.
> 
> ---
> 
> ## Soluções Alternativas e Dicas: Gerenciando Desvios
> 
> Esta seção é um guia prático para o Mestre lidar com as inevitáveis surpresas e desvios que os jogadores trarão para a narrativa, garantindo que a aventura permaneça fluida e gratificante, mesmo quando eles não seguem o caminho planejado.
> 
> **Geral:**
> 
> *   **Pistas Redundantes:** As informações cruciais para o progresso na aventura (como a necessidade de ir para as cavernas, a identidade de Grolbin ou a localização do laboratório final) devem estar disponíveis em múltiplos locais e através de diferentes métodos. Se os jogadores perderem uma pista na investigação, um NPC pode mencioná-la acidentalmente, ou um evento ambiental pode destacá-la.
> *   **Intervenção Externa:** Use os NPCs (Thorne, Roland, Hemlock, talvez até Fitzwilliam/Lena em fuga) ou eventos ambientais (sons estranhos vindos de uma direção, um objeto caindo de um túnel) para fornecer dicas ou redirecionar os jogadores sutilmente se eles estiverem perdidos, presos em um *red herring*, ou se concentrando em detalhes irrelevantes.
> *   **Consequências Ajustáveis:** Seja flexível com a severidade das falhas em testes de perícia ou armadilhas. Uma falha não precisa impedir o progresso; pode simplesmente impor um custo (tempo, recursos, HP) ou levar a uma pista alternativa mais difícil de encontrar. Ajuste CDs ou efeitos de armadilhas "nos bastidores" se o grupo estiver lutando demais.
> 
> **Contingências Específicas por Encontro:**
> 
> *   **Introdução e Encontro 1 (Oficina):**
>     *   **Desconfiança Extrema de Thorne:** Se os jogadores forem hostis ou totalmente desconfiados de Thorne, ele pode ficar ainda mais nervoso e menos cooperativo, mas a urgência da situação ainda o forçará a mostrá-los a oficina. Outros NPCs na cidade (vizinhos de Borin, outros artesãos) podem reforçar a história do desaparecimento e a necessidade de investigar a oficina.
>     *   **Falha Crítica em Encontrar Pistas na Oficina:** Se os jogadores não encontrarem as pistas essenciais na oficina (pegadas, nota, diário), elas podem ser "descobertas" por Roland depois, que, sob pressão, se lembra de algo que viu. Thorne pode mencionar que um guarda viu algo "estranho" (as pegadas) antes de vocês chegarem. O diário pode estar mais visível ou protegido por um mecanismo mais fácil.
>     *   **Foco Excessivo no *Red Herring* de Roland:** Se os jogadores se prenderem na história da peça quebrada, testes de Intuição baixos de Roland confirmam sua honestidade sobre esse detalhe específico. Ele pode implorar que acreditem nele e que o verdadeiro problema é algo "lá fora" que estava assustando Borin.
> *   **Encontro 2 (Cavernas):**
>     *   **Dificuldade em Encontrar a Entrada das Cavernas:** Se os jogadores não associarem o "local de silêncio" às Grutas Sussurrantes, a trilha das pegadas estranhas vista na oficina pode continuar visível fora da cidade, levando-os na direção correta. Hemlock pode ser encontrado mais perto da cidade, resmungando sobre as cavernas.
>     *   **Ignorar ou Assustar Hemlock:** Se os jogadores forem agressivos ou o eremita fugir, ele pode deixar cair uma pista crucial na fuga (um objeto que ele pegou, uma nota que encontrou). Os "sussurros" das cavernas podem se tornar mais direcionais, guiando os jogadores. As pistas ambientais (pegadas, símbolos) nas cavernas podem se tornar mais frequentes e óbvias.
>     *   **Dificuldade com Perigos Naturais/Navegação:** Reduza o dano de quedas ou armadilhas se os jogadores estiverem sofrendo muito. Faça-os encontrar evidências de passagem anterior (pedaços de corda, marcas de sapatos) que facilitam a travessia. Uma das pistas ambientais (como um símbolo rúnico) pode servir como um guia de navegação improvisado.
> *   **Encontro 3 (Rede Oculta):**
>     *   **Falha no Quebra-Cabeça Ambiental:** Se os jogadores não conseguirem resolver o enigma da porta, apresente uma passagem alternativa perigosa (que exige testes de perícia e causa dano). Alternativamente, Fitzwilliam e Lena chegam (talvez mais cedo do que o planejado), sua presença confirma que há algo importante além da porta, e eles (ou seus pertences após um confronto/fuga) podem conter a solução do enigma ou uma chave.
>     *   **Combate Prematuro com Fitzwilliam/Lena:** Se os jogadores atacarem imediatamente, Fitzwilliam e Lena (e capangas, se houver) podem tentar fugir e alertar Grolbin, deixando cair pistas cruciais na fuga. Os capangas podem carregar documentos ou objetos importantes que os jogadores podem recuperar. O ambiente da Rede Oculta pode dificultar a perseguição, permitindo a fuga de alguns, mas à custa de deixar evidências para trás.
>     *   **Falha em Obter Informações Socialmente:** Se os testes sociais falharem, pistas físicas podem ser encontradas no esconderijo improvisado de Fitzwilliam e Lena (caixotes marcados, um mapa improvisado do laboratório, mais metal estranho). Um suborno mais caro pode convencer Lena a revelar algo crucial pelas costas de Fitzwilliam.
> *   **Encontro 4 (Confronto):**
>     *   **Combate Desbalanceado:** Este é o ponto onde o Mestre tem mais flexibilidade nos ajustes "nos bastidores". Se o combate estiver muito fácil, Autômatos extras podem ser ativados por Grolbin, ou o ambiente pode se tornar mais perigoso (faíscas mais frequentes, gás venenoso começando a vazar). Se estiver muito difícil, um Autômato pode falhar, o ambiente pode ter falhas menos frequentes, ou Grolbin pode focar em manobras defensivas/fuga. Reduzir o HP de Grolbin ou dos Autômatos "nos bastidores" é sempre uma opção para evitar uma *party wipe*. Borin (se presente) pode dar dicas mais úteis e imediatas ("O coração é a fraqueza!", "A viga instável!").
>     *   **Morte de Personagem:** Considere opções além da morte permanente. Um personagem pode ser nocauteado em vez de morto, capturado por Grolbin para ser usado em experimentos futuros (um gancho para resgate), ou ter uma morte heroica que dá uma última pista crucial. Uma oportunidade de revivificação pode surgir (talvez exigindo uma mini-quest ou sacrifício). Um personagem temporário (Borin, um capanga desertor, um autômato reprogramado) pode ser oferecido para a batalha restante.
>     *   **Grolbin Escapa:** Se Grolbin conseguir fugir, ele se torna um antagonista recorrente. Ele pode deixar para trás o "coração mecânico" e anotações essenciais, garantindo que os jogadores ainda resolvam o mistério imediato, mas com a ameaça de seu retorno. Ele pode ativar uma última armadilha no laboratório ao fugir.
> 
> **Dicas Rápidas Adicionais:**
> 
> *   **Ataque a NPCs "Inofensivos":** Se os jogadores atacarem NPCs que não são uma ameaça imediata (Roland, Hemlock) sem justificativa narrativa, o NPC fugirá, se defenderá (se puder) ou tentará alertar a guarda. Essa ação deve ter consequências sociais em Oakhaven – testes sociais futuros podem ser mais difíceis, e a guarda pode ficar desconfiada deles. As pistas que o NPC possuía podem ser encontradas em seu corpo (se derrotado) ou em seu esconderijo.
> *   **Grupo se Divide no Encontro 4:** Se os jogadores se dividirem no laboratório perigoso, Grolbin e seus autômatos devem explorar essa vantagem, focando fogo em alvos isolados. O ambiente perigoso também pode dificultar a reunião do grupo.
> 
> A chave final para o sucesso do Mestre é a flexibilidade e a capacidade de improvisar. Tenha as pistas cruciais disponíveis de várias formas e esteja pronto para adaptar a narrativa, os encontros e as consequências às escolhas, sucessos e fracassos dos seus jogadores. Eles não seguirão o script, e é aí que a verdadeira diversão de D&D acontece. Sua estrutura detalhada fornece uma espinha dorsal robusta, mas a carne da história será criada na interação com os jogadores.
> 
> ---
> 
> Com este guia completo, organizado e ricamente detalhado, você está mais do que preparado para conduzir "O Enigma do Artesão Desaparecido". Que seus jogadores mergulhem no mistério, superem os desafios e descubram a verdade por trás do sumiço de Borin Stonehand! Boa sorte na sua sessão, Mestre!


--------------------------------------------------------------
